In [65]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from collections import defaultdict as dd
from itertools import chain
from collections import namedtuple

import os
import pickle
import shelve
import unicodedata
import nltk

import time
from termcolor import colored

from tqdm import trange, tqdm, tqdm_notebook

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import linear_kernel

In [12]:
Article = namedtuple("Article", ["title", "header", "content"])

In [ ]:
# with open(r'dane\fp_wiki.txt', 'r', encoding='utf8') as f:
#     wikipedia_raw = f.read()
# articles_raw = wikipedia_raw.split('\n\n')

# wikipedia_db = []
# for article_raw in tqdm(articles_raw):
#     article_lines = article_raw.split('\n')

#     if len(article_lines) < 2:
#         continue

#     title = article_lines[0][7:]
#     header = article_lines[1]
#     content = '\n'.join(article_lines[2:])
#     if len(content) == 0:
#         continue

#     article = Article(title, header, content)
#     wikipedia_db.append(article)

In [13]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/wikipedia.pickle', 'wb') as file:
#     pickle.dump(wikipedia_db, file, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'.cache/wikipedia.pickle', 'rb') as file:
    wikipedia_db = pickle.load(file)

In [38]:
# stop_words = set()
# for line in open("dane\stop_words.txt", 'r', encoding='utf8'):
#     stop_words.add(line[:-1])
# punctuation = set(["'", "(", ")", "-", "!", "[", "]", "–", "", "&", "?", "''", ":", ';', '``', '.', '``', '„', '”', ','])

In [20]:
title_list_raw = []
for article in tqdm_notebook(wikipedia_db):
    title_list_raw.append(article.title)
title_list = [x.lower().replace(" ","") for x in title_list_raw]

In [41]:
# short_wiki = []
# for art in tqdm_notebook(wikipedia_db):
#     good_words = []
    
#     title = nltk.word_tokenize(art.title)
#     for word in title:
#         word = word.lower()
#         if word in stop_words or word in punctuation:
#             continue
#         good_words.append(word)
        
#     content = nltk.word_tokenize(art.content)
#     for word in content[:100]:
#         word = word.lower()
#         if word in stop_words or word in punctuation:
#             continue
#         good_words.append(word)
        
#     short_article = (good_words)[:50]
#     short_wiki.append(short_article)

In [115]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/short_wiki.pickle', 'wb') as file:
#     pickle.dump(short_wiki, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'.cache/short_wiki.pickle', 'rb') as file:
    short_wiki = pickle.load(file)

In [47]:
short_wiki_conc = []
for article in tqdm_notebook(short_wiki):
    article = ' '.join(article)
    short_wiki_conc.append(article)

In [86]:
# vectorizer = TfidfVectorizer(lowercase=False)
# tfidfs = vectorizer.fit_transform(short_wiki_conc)

In [89]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/tfidfs.pickle', 'wb') as file:
#     pickle.dump(tfidfs, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'.cache/tfidfs.pickle', 'rb') as file:
    tfidfs = pickle.load(file)

In [91]:
def get_ngrams(sentence, n=2):
    ngram = []
    for it in range(len(sentence) - n + 1):
        ngram.append(sentence[it : it + n])
    return ngram

In [92]:
def title_search(q, bdr=5, n=2):
    query = q.lower().replace(" ","")
    query_len = len(query)
    
    grams = get_ngrams(query, n)
    
    candidates_long = [x for x in title_list if (len(x) >= (query_len - bdr) and len(x) <= (query_len + bdr))]

    final_score = []
    for title in candidates_long:
        title_score = 0
        for gram in grams:
            if gram in title:
                title_score += 1
        final_score.append(title_score)
    
    max_val = final_score[np.argmax(final_score)]
    best_fits = [i for i, x in enumerate(final_score) if x == max_val]
    
    if len(best_fits) > 1:
        best_titles =  sorted([(candidates_long[i], i) for i in best_fits], key=lambda x: len(x[0]))
        best_trimmed = [(title[: title.find('(')], i) for title, i in best_titles]
        
        init_len = len(best_trimmed[0][0])
        i = 1
        while i < len(best_trimmed) and len(best_trimmed[i][0]) == init_len:
            i += 1
        
        if i > 1:
            best_earliest = sorted(best_trimmed[:i], key=lambda x: x[1])
            best_fits = [best_earliest[0][1]]
            
            j = 1
            while j < i and best_earliest[j][0] == best_earliest[j-1][0]:
                j += 1
            i = j
            
        best_fits = [i for _, i in best_trimmed[:i]]
    best_fits = [title_list.index(np.array(candidates_long)[i]) for i in best_fits]
    
    return best_fits

In [113]:
def find_similar(query, n_display=5):
    i = title_search(query)[0]
    cosine_similarities = linear_kernel(tfidfs[i:i+1], tfidfs).flatten()
    
    print(f'QUERY: {query}\n')
    for i in np.argsort(cosine_similarities, )[-n_display:]:
        art = wikipedia_db[i]
        print(art.title + '\n')
        print(art.content + '\n')
        print('-' * 100 + '\n')

In [114]:
find_similar('właasadcapierccienai', 10)

QUERY: właasadcapierccienai

Władca Pierścieni (trylogia filmowa)

Władca Pierścieni ( ang . `` The Lord of the Rings '' ) – trzyczęściowe dzieło filmowe z gatunku fantasy nakręcone w oparciu o powieść J.R.R . Tolkiena pod tym samym tytułem . Reżyserem filmu jest Peter Jackson , który wraz ze swoją żoną Frances Walsh i Philippą Boyens napisał także scenariusz .
Niemal wszystkie sceny powstały w czasie osiemnastu miesięcy ciągłej produkcji w plenerach Nowej Zelandii . Skorzystano również z pomocy 10 000 nowozelandzkich kibiców krykieta , którzy w przerwie jednego z meczów pomogli przy nagraniu odgłosów armii orków .

----------------------------------------------------------------------------------------------------

Powrót króla (film 1980)

Powrót króla ( ang . `` The Return of the King '' ) – film animowany z 1980 roku produkcji Rankin/Bass i Topcraft . Jest to filmowa adaptacja powieści Władca Pierścieni . Dwie Wieże oraz Władca Pierścieni : Powrót króla . Film jest często uważany z